In [ ]:
# 相关库导入
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
plt.rcParams['font.sans-serif']=['SimHei']  # 防止中文出现乱码
plt.rcParams['axes.unicode_minus'] = False  # 防止中文出现乱码

In [ ]:
# 图像导入
#img = cv2.imread('Images/lena.jpeg',1)
#img = cv2.imread('Images/girl.jpeg',1)
#img = cv2.imread('Images/woman2.png',1)
#img = cv2.imread('Images/cameraman.jpg',1)
#img = cv2.imread('Images/girl6.jpg',1)
#img = cv2.imread('Images/girl7.jpg',1)     # 终极素材
img = cv2.imread('Images/mopi.jpeg')
#img = cv2.imread('Images/QUT2.jpg',1)
#img = cv2.imread('Images/building.jpg',1)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Image', 1600, 900)
cv2.imshow('Image',img)
cv2.waitKey(0)

In [ ]:
# Gray World 色彩均衡 效果一般
def Gray_World(img):
    height = img.shape[0]
    width = img.shape[1]
    sumR = 0
    sumG = 0
    sumB = 0
    max = 0
    for i in range(0,height):
        for j in range(0,width):
            sumR = sumR + img[i,j,2]
            sumG = sumG + img[i,j,1]
            sumB = sumB + img[i,j,0]              
    avgR = sumR/(height*width)
    avgG = sumG/(height*width)
    avgB = sumB/(height*width)
    print(avgR,avgG,avgB)
    avgGray = (avgR+avgG+avgB)/3
    ar = avgGray/avgR
    ag = avgGray/avgG
    ab = avgGray/avgB
    for i in range(0,height):
        for j in range(0,width):
            img[i,j,2] = int(img[i,j,2]*ar)         # 对于图像中每个像素img[i,j] 调整其R、G、B分量 new Ｒ　＝　old Ｒ ＊ ａｒ
            img[i,j,1] = int(img[i,j,1]*ag)
            img[i,j,0] = int(img[i,j,0]*ab)
            if(img[i,j,2]>max):                     # max为RGB三个分量中的最大值
                max = img[i,j,2]
            if(img[i,j,1]>max):
                max = img[i,j,1]
            if(img[i,j,0]>max):
                max = img[i,j,0]
    factor = max/255                                 # 归一化处理
    print(factor)
    if(factor>1):
        for i in range(0,height):
            for j in range(0,width):
                img[i,j,2] = int(img[i,j,2]/factor)
                img[i,j,1] = int(img[i,j,1]/factor)
                img[i,j,0] = int(img[i,j,0]/factor)
    cv2.imshow('dst',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#Img1 = Gray_World(img)


In [ ]:
# 伽马校正
def Gamma_correction(img,gamma):
    dst = np.array(255 * (img / 255) ** gamma, dtype='uint8')           # 先归一化处理， 再进行非线性变换，最后再映射到[0,255]
    cv2.imshow('src',img)
    cv2.imshow('dst',dst)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Gamma_correction(img,0.9)

In [ ]:
# 直方图均衡化 适用于灰度图像
#def Hist_equalization_Grey(img):
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
equ = cv2.equalizeHist(gray)
cv2.imshow('src',img)
cv2.imshow('equ',equ)

plt.hist(img.ravel(),256,[0,256],label='原图')
plt.figure
plt.hist(equ.ravel(),256,[0,256],label = '均衡化后')
plt.xlabel('灰度值')
plt.ylabel('频数')
plt.legend(loc='upper right')
plt.show()
cv2.waitKey(0)
cv2.destroyAllWindows()

#Hist_equalization_Grey(img)

In [ ]:
# 局部直方图均衡化
# 思路： 将原图分成若干个小块，对每个小块进行直方图均衡化，然后将均衡化后的图像合并起来
# 这种方法主要对于图像直方图不是那么单一的（比如存在多峰情况）图像比较实用
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(300,300))
result = clahe.apply(gray)

#显示图像
plt.subplot(221)
plt.imshow(gray, cmap=plt.cm.gray), plt.axis("off"), plt.title('（a）') 
plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.4, hspace=0.3)       # wspace调整子图的间距
plt.subplot(222)
plt.imshow(equ, cmap=plt.cm.gray), plt.axis("off"), plt.title('（b）') 
plt.subplot(223)
plt.hist(img.ravel(), 256)
plt.title('（c）') 
plt.xlabel('灰度值')
plt.ylabel('频数')
plt.subplot(224)
plt.hist(equ.ravel(), 256,color='#ff7f0e')
plt.title('（d）')
plt.xlabel('灰度值')
plt.ylabel('频数')
plt.show()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 直方图均衡化对比
equ = cv2.equalizeHist(gray)
plt.hist(equ.ravel(),256,[0,256],label="直方图均衡化")
plt.hist(result.ravel(), 256,label="局部直方图均衡化")
plt.xlabel('灰度值')
plt.ylabel('频数')
plt.legend(loc='upper right')       # 添加图例
#plt.title("直方图均衡化对比")
plt.show()

In [ ]:
# 直方图均衡化 适用于彩色图像 
# 思路1：步骤分别对BGR三个通道进行直方图均衡化，然后合并即可
# 思路2：将其分离成HSI空间，其中I是亮度，单独对亮度进行均衡化即可
#图形美化设置

from matplotlib.ticker import FuncFormatter
# 科学计数法形式函数
def formatnum(x, pos):
    return '%.2f×10$^{5}$' % (x/1e5)

def Hist_equalization_RGB(img):
    R = img[:,:,2]
    G = img[:,:,1]
    B = img[:,:,0]
    R_equ = cv2.equalizeHist(R)                 # 均衡化
    G_equ = cv2.equalizeHist(G)         
    B_equ = cv2.equalizeHist(B)
    img_equ = cv2.merge([B_equ,G_equ,R_equ])    # 合并　
    # 显示图像
    plt.figure(0)
    plt.subplot(121)
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    plt.title("原图像")
    plt.subplot(122)
    plt.imshow(img_equ)
    plt.title("对RGB色彩均衡化处理后")

    # cv2.namedWindow('src', cv2.WINDOW_NORMAL)
    # cv2.resizeWindow('src', 1600, 900)
    # cv2.imshow('src',img)
    # cv2.namedWindow('equ', cv2.WINDOW_NORMAL)
    # cv2.resizeWindow('equ', 1600, 900)
    # cv2.imshow('equ',img_equ)                   

    # 以下为直方图绘制代码
    plt.figure(1)

    plt.subplot(2,1,1)
    plt.xlim(0,256)                             # 限制x坐标轴的范围
    plt.hist(R.ravel(),256,[0,256],label='原R分量灰度直方图',color='red')
    plt.xlabel('灰度值')
    plt.ylabel('频数')
    formatter = FuncFormatter(formatnum)
    plt.gca().yaxis.set_major_formatter(formatter)
    plt.legend(loc='upper right')       # 添加图例
    # ax = plt.gca()        # 1e3 形式
    # ax.ticklabel_format(style='sci', scilimits=(-1,2), axis='y')
    # ax.get_yaxis().get_offset_text().set(va='bottom', ha='left')
    # ax.yaxis.get_offset_text().set_fontsize(16)#设置1e6的大小与位置

    plt.subplot(2,1,2)
    plt.hist(R_equ.ravel(),256,[0,256],label='均衡化后直方图',color='OrangeRed')
    plt.xlim(0,256)                             # 限制x坐标轴的范围
    plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.5, hspace=0.3)       # wspace调整子图的间距
    formatter = FuncFormatter(formatnum)
    plt.gca().yaxis.set_major_formatter(formatter)
    plt.xlabel('灰度值')
    plt.ylabel('频数')
    plt.legend(loc='upper right')       # 添加图例
    #plt.yticks(np.linspace(0,4e3,5),fontsize=10)
    #plt.suptitle("R分量直方图均衡化结果图")
    plt.show()
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Hist_equalization_RGB(img)

def Hist_equalization_HSI(img):
    HSI_img = cv2. cvtColor(img,cv2.COLOR_BGR2HSV_FULL)

    H = HSI_img[:,:,0]
    S = HSI_img[:,:,1]
    I = HSI_img[:,:,2]
    I_equ = cv2.equalizeHist(I)
    equ = cv2.merge([H,S,I_equ])
    cv2.namedWindow('src', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('src', 1600, 900)
    cv2.imshow('src',img)
    cv2.namedWindow('equ', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('equ', 1600, 900)
    cv2.imshow('equ',cv2.cvtColor(equ,cv2.COLOR_HSV2BGR))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#Hist_equalization_HSI(img)


def Hist_equalization_HSV(img):
    HSI_img = cv2. cvtColor(img,cv2.COLOR_BGR2HSV)
    H = HSI_img[:,:,0]
    S = HSI_img[:,:,1]
    V = HSI_img[:,:,2]
    V_equ = cv2.equalizeHist(V)
    equ = cv2.merge([H,S,V_equ])
    cv2.namedWindow('src', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('src', 1600, 900)
    cv2.imshow('src',img)
    cv2.namedWindow('equ', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('equ', 1600, 900)
    cv2.imshow('equ',cv2.cvtColor(equ,cv2.COLOR_HSV2BGR))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#Hist_equalization_HSV(img)



In [19]:
# 通过直方图均衡化对Cr分量进行处理
def Hist_equalization_Cr(img):
    ycbcr = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
    Cr = ycbcr[:,:,1]
    #Cr_equ = cv2.equalizeHist(Cr)
    clahe = cv2.createCLAHE(clipLimit=0.6, tileGridSize=(30,30))
    y_equ = ycbcr[:,:,0]
    Cr_equ = clahe.apply(Cr)
    cv2.imshow("1",Cr)
    cv2.imshow("2",Cr_equ)
    ycbcr_equ = cv2.merge([y_equ,Cr_equ,ycbcr[:,:,2]])
    img_equ = cv2.cvtColor(ycbcr_equ, cv2.COLOR_YCR_CB2BGR)
    cv2.imshow("img",img)
    cv2.imshow("equ",img_equ)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img_equ

Hist_equalization_Cr(img)

array([[[ 61,  65, 113],
        [ 56,  60, 108],
        [ 53,  57, 105],
        ...,
        [108, 140, 207],
        [110, 142, 209],
        [104, 136, 203]],

       [[ 59,  63, 111],
        [ 54,  58, 106],
        [ 53,  57, 105],
        ...,
        [112, 144, 211],
        [112, 144, 211],
        [112, 144, 211]],

       [[ 59,  63, 111],
        [ 54,  58, 106],
        [ 53,  57, 105],
        ...,
        [113, 145, 212],
        [111, 143, 210],
        [116, 148, 215]],

       ...,

       [[ 38,  43,  96],
        [ 38,  43,  96],
        [ 38,  43,  96],
        ...,
        [214, 166, 162],
        [214, 166, 162],
        [214, 166, 162]],

       [[ 38,  43,  96],
        [ 38,  43,  96],
        [ 38,  43,  96],
        ...,
        [214, 166, 162],
        [214, 166, 162],
        [215, 167, 163]],

       [[ 38,  43,  96],
        [ 38,  43,  96],
        [ 38,  43,  96],
        ...,
        [214, 166, 162],
        [215, 167, 163],
        [215, 167, 163]]

In [21]:
# 腐蚀 膨胀
def erode(img,kernel):
    height, width = img.shape
    h = kernel.shape[0]
    w = kernel.shape[1]
    erosion = np.zeros((height,width),np.uint8)
    _,Result = cv2.threshold(img,66,1,cv2.THRESH_BINARY)                        # 将图像中的像素映射到0和1 主要调第二个参数值
    for i in range(h//2,height-h//2):                                           # // 代表向下取整整除
        for j in range(h//2,width-h//2):
            if np.sum(Result[i-h//2:i+h//2+1,j-h//2:j+h//2+1]*kernel)< 9:       # *为普通的点乘 与运算
                erosion[i,j] = 0                                                # 前景为白色,若该位置像素点不存在，则另其为0（黑色，背景色）
            else:
                erosion[i,j] = np.max(img[i-h//2:i+h//2+1,j-h//2:j+h//2+1])     # 取模板覆盖中的最大值
    return erosion

def dilate(img,kernel):
    height, width = img.shape
    h = kernel.shape[0]
    w = kernel.shape[1]
    dilation = np.zeros((height,width),np.uint8)
    _,Result = cv2.threshold(img,30,1,cv2.THRESH_BINARY)                        # 将图像中的像素映射到0和1 主要调第二个参数值
    for i in range(h//2,height-h//2):                                           # // 代表向下取整整除
        for j in range(h//2,width-h//2):
            if np.sum(Result[i-h//2:i+h//2+1,j-h//2:j+h//2+1]*kernel)< 1:       # *为普通的点乘 与运算
                dilation[i,j] = 0                                               # 前景为白色
            else:
                dilation[i,j] = np.max(img[i-h//2:i+h//2+1,j-h//2:j+h//2+1])
    return dilation

In [ ]:

# def reference_white(img):
#     img_b = np.sort(img[:, :, 0].reshape( -1))
#     img_g = np.sort(img[:, :, 1].reshape( -1))
#     img_r = np.sort(img[:, :, 2].reshape( -1))
#     num1 = int(img_b.shape[0] * 0.05)
#     num2 = int(img_g.shape[0] * 0.05)
#     num3 = int(img_r.shape[0] * 0.05)
#     img_b_mean = np.mean(img_b[-num1:])
#     img_g_mean = np.mean(img_g[-num2:])
#     img_r_mean = np.mean(img_r[-num3:])
#     img[:, :, 0] = np.uint8(np.minimum(img[:, :, 0] * (255 / img_b_mean), 255))
#     img[:, :, 1] = np.uint8(np.minimum(img[:, :, 1] * (255 / img_g_mean), 255))
#     img[:, :, 2] = np.uint8(np.minimum(img[:, :, 2] * (255 / img_r_mean), 255))
#     return img


In [ ]:
# 参考白算法 知乎找的，没找到源文献
def reference_white(img):
    img_shape = img.shape
    img_b = img[:,:,0].astype('float')
    img_g = img[:,:,1].astype('float')
    img_r = img[:,:,2].astype('float')
    
    img_ycbcr = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
    img_y = img_ycbcr[:,:,0]
    img_y_sort = np.sort(img_y.reshape( -1))
    num = int(img_y_sort.shape[0] * 0.03)
    T1 = img_y_sort[-num]
    T2 = img_y_sort[num]
    
    n1 = 0
    n2 = 0
    img_r_big_sum = 0
    img_g_big_sum = 0
    img_b_big_sum = 0
    img_r_small_sum = 0
    img_g_small_sum = 0
    img_b_small_sum = 0
    
    for i in range(img_shape[0]):
        for j in range(img_shape[1]):
            if img_y[i, j] >= T1:
                img_r_big_sum =+ img_r[i,j]
                img_g_big_sum =+ img_g[i,j]
                img_b_big_sum =+ img_b[i,j]
                n1 =+ 1

            if img_y[i, j] <= T2:
                img_r_small_sum = + img_r[i, j]
                img_g_small_sum = + img_g[i, j]
                img_b_small_sum = + img_b[i, j]
                n2 =+ 1
    
    r_mean1 = img_r_big_sum/n1
    g_mean1 = img_g_big_sum/n1
    b_mean1 = img_b_big_sum/n1
    
    r_mean2 = img_r_small_sum/n2
    g_mean2 = img_g_small_sum/n2
    b_mean2 = img_b_small_sum/n2

    gama_r = -1
    gama_g = -1
    gama_b = -1
    
    print(r_mean2, r_mean1)
    for i in range(img_shape[0]):
        for j in range(img_shape[1]):
            img_r[i,j] = (img_r[i,j] - r_mean2)/(r_mean2 - r_mean1)*gama_r
            img_g[i,j] = (img_g[i,j] - g_mean2)/(g_mean2 - g_mean1)*gama_g
            img_b[i,j] = (img_b[i,j] - b_mean2)/(b_mean2 - b_mean1)*gama_b
            
    img_new = np.zeros(shape=[img_shape[0], img_shape[1], 3])
    img_new[:,:,0] = img_b
    img_new[:,:,1] = img_g
    img_new[:,:,2] = img_r

    img_new = np.minimum(img_new, 1)
    img_new = np.maximum(img_new,0)
    img_new = np.uint8(img_new*255)
    cv2.imshow('img',img)
    cv2.imshow('img_new',img_new)
    cv2.waitKey()
    return img_new

reference_white(img)


In [ ]:
#  改进的YCrCb转换公式 垃圾不好用
ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      # RGB->YCrCb  
(Y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图
B = img[:,:,0]                                     # 获取B通道
G = img[:,:,1]
R = img[:,:,2]
for i in range(imgInfo[0]):
    for j in range(imgInfo[1]):
        if(Y[i,j]<200):
            cr = int((B[i,j]-Y[i,j])*0.713)
            cb = int((B[i,j]-Y[i,j])*0.564)
        else:
            print(R[i,j],Y[i,j])
            cr = ((R[i,j]-Y[i,j])**2*0.713*((-5000/91)*(Y[i,j]-200)**(-2.0)+7))
            print("cr="+str(cr))
            cb = int(-1*(B[i,j]-Y[i,j])**2*0.564*(125*(Y[i,j]-200)**(-2.0)-3))
            print("cb="+str(cb))
img2 = (Y,cr,cb)
img3 =  cv2.cvtColor(img,cv2.COLOR_YCrCb2BGR)
cv2.imshow("IMG",img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 线性动态调整
def Dynamic_adjust_liner(img,th1,th2):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    height = img.shape[0]
    width = img.shape[1]
    dst = img.copy()
    for i in range(0,height):
        for j in range(0,width):
                if(img[i,j]<th1):
                    dst[i,j]= 0
                elif(img[i,j]<th2):
                    dst[i,j]= 255/(th2-th1)*(img[i,j]-th1)
                elif(img[i,j]>=th2):
                     dst[i,j] = 255
    cv2.imshow('dst',dst)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return dst

dst = Dynamic_adjust_liner(img,100,150)

In [ ]:
# 肤色检测 基于YCrCb颜色空间的CrCb分量筛选
def YCrCb_CrCb(img):
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  
    (y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像
    height = imgInfo[0]
    width = imgInfo[1]
    dst = img.copy()
    for i in range(0,height):
        for j in range(0,width):
            if(cr[i,j]>135 and cr[i,j]<175 and cb[i,j]>77 and cb[i,j]<127):
                dst[i,j] = (255,255,255)
            else:
                dst[i,j] = (0,0,0)
                
    cv2.imshow('dst',dst)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return dst

YCrCb_CrCb(img)

In [ ]:
# 肤色检测 基于YCrCb颜色空间的Cr分量+Otsu法阈值分割算法
def YCrCb_OTSU(img):
    start = time.time()
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  
    (y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像

    # 高斯滤波, cr 是待滤波的源图像数据, (5,5)是值窗口大小, 0 是指根据窗口大小来计算高斯函数标准差
    cr1 = cv2.GaussianBlur(cr, (5, 5), 0) # 对cr通道分量进行高斯滤波
    # 根据OTSU算法求图像阈值, 对图像进行二值化
    th, skin = cv2.threshold(cr1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
    end = time.time()
    print("阈值为"+str(th))
    print("耗时"+str(end-start)+"秒")
    cv2.imshow("image CR", cr1)
    cv2.imshow("Skin Cr+OSTU", skin )
    cv2.waitKey(0)
    return skin


#YCrCb_OTSU(img)

In [ ]:
# 绘制YCrCb颜色空间图像
ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  
(y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像

fig, axs = plt.subplots(1, 4, figsize=(10,10))          # 创建1X4的网格，其中每个子图大小为7X7 英寸：

plt.subplot(1,4,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("原图")                                       # 为第一个子图添加标题
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.show

plt.subplot(1,4,2)
plt.imshow(y,cmap='Greys_r')
plt.title("y分量")                                       
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.show

plt.subplot(1,4,3)
plt.imshow(cr,cmap='Greys_r')
plt.title("Cr分量")                                       
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.show

plt.subplot(1,4,4)
plt.imshow(cb,cmap='Greys_r')
plt.title("Cb分量")  
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.show



In [ ]:
# 像素分布概率直方图
def showimg_pro_eachgray(img):                                          # 显示每一个灰度级的像素数概率
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                          # 灰度化处理
          
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])            # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                          # 获得每个灰度级中像素数占总像素数比例则我们获得了pi的一个向量
    x = np.linspace(0, 255, 256)                                        # 横坐标灰度级别
    plt.subplot(2,1,1)
    plt.imshow(img,'gray')                                              # 如果不加gray就会出现以前hsv与rgb的问题
    plt.title('origin image')
    plt.colorbar(orientation="horizontal")                              # 水平放置颜色条
    plt.subplot(2,1,2)
    plt.bar(x, p.ravel(), 0.9, alpha=1, color='b')
    plt.title('Histogram of the probablity for each gray level',y=-0.4) # 调节标题上下移动的方法
    plt.show()

showimg_pro_eachgray(img)


In [22]:
# 基于手动编程的OTSU 实现的YCrCb分割 
def OTSU(img):
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                             # 若待划分的图像是单通道，则无需转为灰度图
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])                # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                              # 获得每个灰度级中像素数占总像素数比例则我们获得了p_i的一个向量
    x=np.linspace(1,256,256)                                                # 灰度级像素范围定义，若定义为0～255，则求平均灰度级时会忽略第一个数据，所以我们定义为从1到256                                                 
    maxvar=0
    mG = np.sum(x*p)                                                        # 平均灰度级
    th=0
    step = 0
    for k1 in range(1,256):                                                 # 暴力搜索
        p1 = np.sum(p[0:k1])
        p2 = np.sum(p[k1:256])
        m1 = np.dot(x[0:k1],p[0:k1])/p1
        m2 = np.dot(x[k1:256],p[k1:256])/p2
        vars = p1*p2*(m1-m2)**2
        if(vars>maxvar):
            maxvar = vars
            th = k1
            step = step+1
            print("第"+str(step)+"次找到最佳阈值"+str(th)+","+"最大类间方差为"+str(maxvar))
    return th,maxvar 
print("最佳阈值是"+str(OTSU(img)[0]),"最大方差为"+str(OTSU(img)[1]))


# 基于手动编程的OTSU 实现的YCrCb分割                  # 凸显改进算法的优越性，使用mopi.jpeg例子，能良好体现出改进之后的算法，具有更强的识别效果，在处理肤色相近，像素值集中的图像时更占上风
def YCrCb_OTSU(img):
    start  = time.time()                               # 引入计时模块
    img = Hist_equalization_Cr(img)
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  有没有转换公式呢？
    (y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像
    cr1 = cv2.GaussianBlur(cr, (5, 5), 0) # # 高斯滤波, cr 是待滤波的源图像数据, (5,5)是值窗口大小, 0 是指根据窗口大小来计算高斯函数标准差
    th = OTSU(cr1)[0]
    skin = cr1.copy()
    # 根据OTSU算法求图像阈值, 对图像进行二值化 获得人脸部分（白色）,会发现因为手部与脸部肤色相近，故被认定为前景（人脸部分）
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            if(cr1[i,j]< th):           
                skin[i,j]= 0  
            else:  
                skin[i,j] = 255

    # # 原图的人脸部分
    # for i in range(0,img.shape[ßß0]):
    #     for j in range(0,img.shape[1]): 
    #         if(skin[i,j]==0):
    #             img[i,j] = 0
    # cv2.imshow("SKin",img)
    end = time.time()
    print("耗时"+str(end-start)+"s")
    # 显示图像
    cv2.imshow("Skin Cr+OSTU_Hand", skin) # 因为通过对YCrCb空间分割再还原RGB比较麻烦
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
YCrCb_OTSU(img)

第1次找到最佳阈值1,最大类间方差为[21.49265152]
第2次找到最佳阈值2,最大类间方差为[28.60677287]
第3次找到最佳阈值3,最大类间方差为[36.34682459]
第4次找到最佳阈值4,最大类间方差为[45.03307132]
第5次找到最佳阈值5,最大类间方差为[55.50225459]
第6次找到最佳阈值6,最大类间方差为[67.07035141]
第7次找到最佳阈值7,最大类间方差为[80.02348912]
第8次找到最佳阈值8,最大类间方差为[93.62067965]
第9次找到最佳阈值9,最大类间方差为[106.9080899]
第10次找到最佳阈值10,最大类间方差为[121.15393796]
第11次找到最佳阈值11,最大类间方差为[136.14607301]
第12次找到最佳阈值12,最大类间方差为[151.53217036]
第13次找到最佳阈值13,最大类间方差为[167.50235512]
第14次找到最佳阈值14,最大类间方差为[183.80305796]
第15次找到最佳阈值15,最大类间方差为[199.11770037]
第16次找到最佳阈值16,最大类间方差为[215.65910801]
第17次找到最佳阈值17,最大类间方差为[231.30850849]
第18次找到最佳阈值18,最大类间方差为[247.02546784]
第19次找到最佳阈值19,最大类间方差为[262.36655873]
第20次找到最佳阈值20,最大类间方差为[278.66326728]
第21次找到最佳阈值21,最大类间方差为[294.44601093]
第22次找到最佳阈值22,最大类间方差为[310.37876774]
第23次找到最佳阈值23,最大类间方差为[327.62971584]
第24次找到最佳阈值24,最大类间方差为[344.3554747]
第25次找到最佳阈值25,最大类间方差为[361.01459023]
第26次找到最佳阈值26,最大类间方差为[377.07501596]
第27次找到最佳阈值27,最大类间方差为[393.9586432]
第28次找到最佳阈值28,最大类间方差为[411.17167494]
第29次找到最佳阈值29,最大类间方差为[428.35365403]
第30次找到最佳阈值30,最

C:\Users\Yehui_PC\AppData\Local\Temp\ipykernel_12200\3192852660.py:14: RuntimeWarning: invalid value encountered in divide
  m1 = np.dot(x[0:k1],p[0:k1])/p1
C:\Users\Yehui_PC\AppData\Local\Temp\ipykernel_12200\3192852660.py:15: RuntimeWarning: invalid value encountered in divide
  m2 = np.dot(x[k1:256],p[k1:256])/p2


第1次找到最佳阈值117,最大类间方差为[0.02829512]
第2次找到最佳阈值118,最大类间方差为[0.50071363]
第3次找到最佳阈值119,最大类间方差为[1.35468985]
第4次找到最佳阈值120,最大类间方差为[2.73711452]
第5次找到最佳阈值121,最大类间方差为[5.72732722]
第6次找到最佳阈值122,最大类间方差为[12.62638003]
第7次找到最佳阈值123,最大类间方差为[17.3943432]
第8次找到最佳阈值124,最大类间方差为[20.11070033]
第9次找到最佳阈值125,最大类间方差为[22.13337042]
第10次找到最佳阈值126,最大类间方差为[25.22359601]
第11次找到最佳阈值127,最大类间方差为[25.87255531]
第12次找到最佳阈值128,最大类间方差为[26.31343633]
第13次找到最佳阈值129,最大类间方差为[26.87469751]
第14次找到最佳阈值130,最大类间方差为[27.68572821]
第15次找到最佳阈值131,最大类间方差为[28.49760286]
第16次找到最佳阈值132,最大类间方差为[30.00606179]
第17次找到最佳阈值133,最大类间方差为[31.17215174]
第18次找到最佳阈值134,最大类间方差为[32.29148323]
第19次找到最佳阈值135,最大类间方差为[33.46388078]
第20次找到最佳阈值136,最大类间方差为[34.94134096]
第21次找到最佳阈值137,最大类间方差为[36.6151495]
第22次找到最佳阈值138,最大类间方差为[38.98247197]
第23次找到最佳阈值139,最大类间方差为[41.44317588]
第24次找到最佳阈值140,最大类间方差为[43.9752268]
第25次找到最佳阈值141,最大类间方差为[45.75905684]
第26次找到最佳阈值142,最大类间方差为[47.05775807]
第27次找到最佳阈值143,最大类间方差为[48.02135728]
第28次找到最佳阈值144,最大类间方差为[49.89118422]
第29次找到最佳阈值145,最大类间方差为[51.22205044]
第3

In [ ]:
# OTSU法实现图像分割
def OTSU_Seg(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                             # 若待划分的图像是单通道，则无需转为灰度图
    _, obj = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # cv2.imshow("img",img)
    # cv2.imshow("Use_OTSU_Seg",obj)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return obj

obj = OTSU_Seg(img)  
#figure()
fig, axs = plt.subplots(1, 2, figsize=(14,14))          # 创建1X2的网格，其中每个子图大小为14X14 英寸：
plt.subplot(1,2,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("Lena.jpg")                                       # 为第一个子图添加标题

plt.subplot(1,2,2)                                      # 占据第二个位置
plt.imshow(obj,cmap='Greys_r')                          # plt默认显示三通道图像
plt.title("调用threshold函数+OTSU阈值分割后")                              # 为第一个子图添加标题


In [ ]:
# 手撸OTSU实现图像分割
def OTSU_Seg_Hand(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                             # 若待划分的图像是单通道，则无需转为灰度图
    th = OTSU(img)[0]
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):     
            if(img[i,j]<th):
                img[i,j]=0
            else:
                img[i,j]=255
    return img 
obj = OTSU_Seg_Hand(img)  
fig, axs = plt.subplots(1, 2, figsize=(14,14))          # 创建1X2的网格，其中每个子图大小为14X14 英寸：
plt.subplot(1,2,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("Cameraman.jpg")                                       # 为第一个子图添加标题

plt.subplot(1,2,2)                                      # 占据第二个位置
plt.imshow(obj,cmap='Greys_r')                          # plt默认显示三通道图像
plt.title("使用手撸OTSU法阈值分割后")                              # 为第一个子图添加标题

In [23]:
# 基于手撸的OTSU_IMPROVE 实现的YCrCb分割                  
# 凸显改进算法的优越性，使用mopi.jpeg例子，能良好体现出改进之后的算法，具有更强的识别效果，在处理肤色相近，像素值集中的图像时更占上风

#  手撸实现最大类间类内方差比 
def OTSU_IMPROVE(img):
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                          # 若待划分的图像是单通道，则无需转为灰度图
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])                # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                              # 获得每个灰度级中像素数占总像素数比例则我们获得了p_i的一个向量
    x=np.linspace(1,256,256)                                                # 灰度级像素范围定义，若定义为0～255，则求平均灰度级时会忽略第一个数据，所以我们定义为从1到256                                                 
    sigma_b = 0                                                             # 类间方差
    sigma_in = 0                                                            # 类内方差
    maxvar = 0                                                              # 最大类间类内方差比
    mG = np.sum(x*p)                                                        # 平均灰度级
    th=0
    step = 0
    for k1 in range(1,256):                                                 # 暴力搜索
        p1 = np.sum(p[0:k1])
        p2 = np.sum(p[k1:256])
        m1 = np.dot(x[0:k1],p[0:k1])/p1
        m2 = np.dot(x[k1:256],p[k1:256])/p2
        vars1 = p1*p2*(m1-m2)**2
        if(vars1>sigma_b):
            sigma_b = vars1
        vars2 = np.sum(p[0:k1]*(x[0:k1]-m1)**2)+np.sum(p[k1:256]*(x[k1:256]-m2)**2)
        if(vars2>sigma_in):
            sigma_in = vars2
        if(sigma_in>0):
            vars = sigma_b/sigma_in
        else:
            vars = 0
        if(vars>maxvar):
            maxvar = vars
            th = k1
            step = step + 1
            print("第"+str(step)+"次找到最佳阈值"+str(th)+","+"最大类间方差为"+str(maxvar))
    return th,maxvar 

print("最佳阈值是"+str(OTSU_IMPROVE(img)[0]),"最大方差比为"+str(OTSU_IMPROVE(img)[1]))    

        
def YCrCb_OTSU_IMPROVE(img):
    start  = time.time()                               # 引入计时模块
    img = Hist_equalization_Cr(img)
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  有没有转换公式呢？
    (y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像
    cr1 = cv2.GaussianBlur(cr, (5, 5), 0) # # 高斯滤波, cr 是待滤波的源图像数据, (5,5)是值窗口大小, 0 是指根据窗口大小来计算高斯函数标准差
    th = OTSU_IMPROVE(cr1)[0]
    skin = cr1.copy()
    # 根据OTSU算法求图像阈值, 对图像进行二值化 获得人脸部分（白色）,会发现因为手部与脸部肤色相近，故被认定为前景（人脸部分）
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            if(cr1[i,j]< th):           
                skin[i,j]= 0  
            else:  
                skin[i,j] = 255

    # # 原图的人脸部分
    # for i in range(0,img.shape[ßß0]):
    #     for j in range(0,img.shape[1]): 
    #         if(skin[i,j]==0):
    #             img[i,j] = 0
    # cv2.imshow("SKin",img)
    end = time.time()
    print("耗时"+str(end-start)+"s")
    #显示图像
    cv2.imshow("Skin Cr+OSTU_IMPROVE_Hand", skin) # 因为通过对YCrCb空间分割再还原RGB比较麻烦
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return skin

kernel = np.ones((3,3),np.uint8)   # 定义一个3*3的结构元素
skin = YCrCb_OTSU_IMPROVE(img)
cv2.imshow("skin.jpg",skin)
skin1 = dilate(skin,kernel)
cv2.imshow("skin1.jpg",skin1)
cv2.waitKey(0)
cv2.destroyAllWindows()


第1次找到最佳阈值1,最大类间方差为[1.41485331e-05]
第2次找到最佳阈值2,最大类间方差为[1.88317329e-05]
第3次找到最佳阈值3,最大类间方差为[2.39269804e-05]
第4次找到最佳阈值4,最大类间方差为[2.96451045e-05]
第5次找到最佳阈值5,最大类间方差为[3.65369292e-05]
第6次找到最佳阈值6,最大类间方差为[4.41521646e-05]
第7次找到最佳阈值7,最大类间方差为[5.26791673e-05]
第8次找到最佳阈值8,最大类间方差为[6.16301476e-05]
第9次找到最佳阈值9,最大类间方差为[7.03772007e-05]
第10次找到最佳阈值10,最大类间方差为[7.97551899e-05]
第11次找到最佳阈值11,最大类间方差为[8.9624457e-05]
第12次找到最佳阈值12,最大类间方差为[9.97530681e-05]
第13次找到最佳阈值13,最大类间方差为[0.00011027]
第14次找到最佳阈值14,最大类间方差为[0.000121]
第15次找到最佳阈值15,最大类间方差为[0.00013108]
第16次找到最佳阈值16,最大类间方差为[0.00014197]
第17次找到最佳阈值17,最大类间方差为[0.00015227]
第18次找到最佳阈值18,最大类间方差为[0.00016262]
第19次找到最佳阈值19,最大类间方差为[0.00017271]
第20次找到最佳阈值20,最大类间方差为[0.00018344]
第21次找到最佳阈值21,最大类间方差为[0.00019383]
第22次找到最佳阈值22,最大类间方差为[0.00020432]
第23次找到最佳阈值23,最大类间方差为[0.00021568]
第24次找到最佳阈值24,最大类间方差为[0.00022669]
第25次找到最佳阈值25,最大类间方差为[0.00023765]
第26次找到最佳阈值26,最大类间方差为[0.00024823]
第27次找到最佳阈值27,最大类间方差为[0.00025934]
第28次找到最佳阈值28,最大类间方差为[0.00027067]
第29次找到最佳阈值29,最大类间方差为[0.00028198]
第30次找到最佳阈值30,最大类

C:\Users\Yehui_PC\AppData\Local\Temp\ipykernel_12200\1733875761.py:19: RuntimeWarning: invalid value encountered in divide
  m1 = np.dot(x[0:k1],p[0:k1])/p1
C:\Users\Yehui_PC\AppData\Local\Temp\ipykernel_12200\1733875761.py:20: RuntimeWarning: invalid value encountered in divide
  m2 = np.dot(x[k1:256],p[k1:256])/p2


第1次找到最佳阈值117,最大类间方差为[9.86514084e-08]
第2次找到最佳阈值118,最大类间方差为[1.74574636e-06]
第3次找到最佳阈值119,最大类间方差为[4.72314861e-06]
第4次找到最佳阈值120,最大类间方差为[9.54299512e-06]
第5次找到最佳阈值121,最大类间方差为[1.99684213e-05]
第6次找到最佳阈值122,最大类间方差为[4.40220832e-05]
第7次找到最佳阈值123,最大类间方差为[6.06456658e-05]
第8次找到最佳阈值124,最大类间方差为[7.01162899e-05]
第9次找到最佳阈值125,最大类间方差为[7.71683627e-05]
第10次找到最佳阈值126,最大类间方差为[8.79424854e-05]
第11次找到最佳阈值127,最大类间方差为[9.02050928e-05]
第12次找到最佳阈值128,最大类间方差为[9.17422318e-05]
第13次找到最佳阈值129,最大类间方差为[9.36990782e-05]
第14次找到最佳阈值130,最大类间方差为[9.65267502e-05]
第15次找到最佳阈值131,最大类间方差为[9.93573646e-05]
第16次找到最佳阈值132,最大类间方差为[0.00010462]
第17次找到最佳阈值133,最大类间方差为[0.00010868]
第18次找到最佳阈值134,最大类间方差为[0.00011258]
第19次找到最佳阈值135,最大类间方差为[0.00011667]
第20次找到最佳阈值136,最大类间方差为[0.00012182]
第21次找到最佳阈值137,最大类间方差为[0.00012766]
第22次找到最佳阈值138,最大类间方差为[0.00013591]
第23次找到最佳阈值139,最大类间方差为[0.00014449]
第24次找到最佳阈值140,最大类间方差为[0.00015332]
第25次找到最佳阈值141,最大类间方差为[0.00015954]
第26次找到最佳阈值142,最大类间方差为[0.00016407]
第27次找到最佳阈值143,最大类间方差为[0.00016743]
第28次找到最佳阈值144,最大类间方差为[0.0001739

In [ ]:
# 多阈值分割的OTSU改进算法    用于处理口红色号较深，将口红认定为前景的情况
def OTSU_Mult_IMPROVE(img):
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])                # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                              # 获得每个灰度级中像素数占总像素数比例则我们获得了p_i的一个向量
    x=np.linspace(1,256,256)                                                # 灰度级像素范围定义，若定义为0～255，则求平均灰度级时会忽略第一个数据，所以我们定义为从1到256                                                 
    sigma_b = 0                                                             # 类间方差
    sigma_in = 0                                                            # 类内方差
    maxvar = 0                                                              # 最大类间类内方差
    mu_T = np.sum(x*p)                                                        # 平均灰度级
    th1=0
    th2=0
    step = 0
    for k1 in range(1,256):                                                 # 暴力搜索
        for k2 in range(1,256):
                a0 = np.sum(p[0:k1])
                a1 = np.sum(p[k1:256])
                a2 = 1-a0-a1
                mu_0 = np.dot(x[0:k1],p[0:k1])/a0
                mu_1 = np.dot(x[k1:k2],p[k1:k2])/a1
                mu_2 = np.dot(x[k2:256],p[k2:256])/a2
                vars1 = a0*(mu_0-mu_T)**2+a1*(mu_1-mu_T)**2+a2*(mu_2-mu_T)**2

    return th1,th2,maxvar


In [ ]:
# YCrCb_OTSU及其改进算法的作图
skin1 = YCrCb_OTSU(img)
skin2=  YCrCb_OTSU_IMPROVE(img)
fig, axs = plt.subplots(1, 3, figsize=(14,14))          # 创建1X2的网格，其中每个子图大小为14X14 英寸：
plt.subplot(1,3,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("原图")                                       # 为第一个子图添加标题

plt.subplot(1,3,2)                                      # 占据第二个位置
plt.imshow(skin1,cmap='Greys_r')                         
plt.title("使用手撸OTSU法阈值分割后")                          

plt.subplot(1,3,3)                                      # 占据第三个位置
plt.imshow(skin2,cmap='Greys_r')                        
plt.title("使用YCrCb+改进OTSU法阈值分割后")


In [ ]:
# 手撸OTSU_IMPROVE实现图像分割
def OTSU_Seg_IMPROVE(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                             # 若待划分的图像是单通道，则无需转为灰度图
    th = OTSU_IMPROVE(img)[0]
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):     
            if(img[i,j]<th):
                img[i,j]=0
            else:
                img[i,j]=255
    return img 
obj = OTSU_Seg_IMPROVE(img)  
fig, axs = plt.subplots(1, 2, figsize=(14,14))          # 创建1X2的网格，其中每个子图大小为14X14 英寸：
plt.subplot(1,2,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("Cameraman.jpg")                                       # 为第一个子图添加标题

plt.subplot(1,2,2)                                      # 占据第二个位置
plt.imshow(obj,cmap='Greys_r')                          # plt默认显示三通道图像
plt.title("使用改进的OTSU法阈值分割后")                              # 为第一个子图添加标题

In [ ]:
# 二次多项式模式检测
def Quad_poly(img):
    Info = img.shape
    height = Info[0]
    width = Info[1]
    Channels = Info[2]
    # 创建一个与原图像大小相同的零数组
    skin = np.zeros((height,width),np.uint8)
    for i in range(0,height):
        for j in range(0,width):
            B = img[i,j,0]
            G = img[i,j,1]
            R = img[i,j,2]
            if(R-G>=20):
                if(G>B):
                    sum = R+G+B
                    t1 = 100*R-33*sum
                    t2 = 100*G-33*sum
                    if((t1**2+t2**2)>=4*sum*sum):
                        T1 = 10000*G*sum
                        lower = -7760*R**2 + 5601*R*sum + 1766*sum**2
                        if(T1>lower):
                            upper = -13767*R**2 + 10743*R*sum + 1452*sum**2
                            if(T1<upper):
                                skin[i,j] = 255
    cv2.imshow('skin2',skin)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return skin
#skin = Quad_poly(img)
# 效果垃圾 笑果还行     人脸面部会反光，导致反光区域的RGB值偏高，会被筛选出人脸区域，而人脸轮廓例如脸颊、腮部亮度比较稳定，会被识别为人脸区域


In [ ]:
# 基于HSV颜色空间H，S，V范围筛选法
def HSV(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # 把图像转换到HSV色域
    (h, s, v) = cv2.split(hsv) # 图像分割, 分别获取h, s, v 通道分量图像
    skin = np.zeros(h.shape, np.uint8)  # 根据源图像的大小创建一个全0的矩阵,用于保存图像数据
    (height, width) = img.shape[0:2] # 获取源图像数据的长和宽

    # 遍历图像, 判断HSV通道的数值, 如果在指定范围中, 则置把新图像的点设为255,否则设为0
    for i in  range(0, height):
        for j in  range(0, width):
            if (h[i][j] >  2) and (h[i][j] <  15) and (s[i][j] >  35) and (s[i][j] <  255) and (v[i][j] >  60) and (v[i][j] <  255):
                skin[i][j] =  255
            else:
                skin[i][j] =  0
    cv2.imshow('skin',skin)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return skin

# hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # 把图像转换到HSV色域
# cv2.imshow('hsv',hsv)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
#HSV(img)
    


In [ ]:
# 双边滤波
dst1 = cv2.bilateralFilter(img,15,35,35) # 邻域半径为15 sigma_d为空间高斯函数标准差，sigma_r为灰度值相似性高斯函数标准差
#cv2.imshow('dst1',dst1)


In [ ]:
# 磨皮算法 
# 通过上述算法获得皮肤区域，遍历双边滤波后的矩阵，将非皮肤区域还原
def skin_(dst1,img,skin1):       #dst1 为双边滤波后的矩阵  img 为原图 skin1 为皮肤区域
    dst2 = dst1.copy()
    for i in range(0,height):   
        for j in range(0,width):
            if(skin1[i,j] == 0):
                dst2[i,j] = img[i,j]

    cv2.imshow('dst2',dst2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()